# tf-idf

In [1]:
import os
x = os.environ["VIRTUAL_ENV"] + "/.."
%cd {x}
del x

/home/jack/docos/uni/dialogue/assessment/hyperpartisan/processing


In [2]:
import pandas as pd
import numpy as np
import pickle

from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

import hyper.eval as evl
from hyper.transformers import StaticTransformer

In [3]:
# FIX: I'm lowering here, but the saved X_transformed data hasn't been lowered

In [4]:
data = pd.read_csv(
    "../data/processed.csv",
    sep="\t",
    dtype={"content": "string", "label": bool}
)
data["content"] = data["content"].str.lower()
X = data["content"]
y = data["label"]

In [5]:
vectorizer = TfidfVectorizer()

In [6]:
pipeline = make_pipeline(
    vectorizer, 
    LogisticRegression(max_iter=300)
)

In [7]:
evl.report_algorithm(X, y, pipeline)

{'acc': [0.7643410852713178, 0.031239444464511462],
 'prec': [0.7826517273576098, 0.04778625921401738],
 'rec': [0.5, 0.07744949969994928],
 'f1': [0.6074292005846245, 0.06682565873424605]}

So by itself, it's not too bad

Let's try it with the semantic features from before.

In [8]:
with open("../data/transformed/spacy_flair.pickle", "rb") as file:
    X_transformed = pickle.load(file)

In [9]:
static_transformer = StaticTransformer(X_transformed)

In [10]:
union = FeatureUnion([
    ("tfidf", TfidfVectorizer()),
    ("static", static_transformer),
])

In [11]:
pipeline = make_pipeline(
    union,
    LogisticRegression(max_iter=300, random_state=13, class_weight="balanced")
)

In [12]:
evl.report_algorithm(X, y, pipeline)

{'acc': [0.8093023255813951, 0.03237304343848232],
 'prec': [0.7277627337201805, 0.04604795489643126],
 'rec': [0.7776595744680851, 0.06301289725174804],
 'f1': [0.7502900153262495, 0.04301680686434799]}